Importing necessary libraries

In [2]:
import os

if "CUDA_VISIBLE_DEVICES" not in os.environ:
    os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(
            gpu, True
        )  # Limiting the memory growth
    logical_gpus = tf.config.experimental.list_logical_devices("GPU")
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

from pathlib import Path
import numpy as np
import pandas as pd


import os
import numpy as np
import random
import os 
import sklearn 
import tensorflow as tf
import optuna 
#import sentence_transformers 
import rdkit

1 Physical GPUs, 1 Logical GPUs


In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
from tensorflow.keras.activations import selu,gelu
from optuna.samplers import CmaEsSampler
import os
import numpy as np
import random

from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,QuantileTransformer,PowerTransformer,MaxAbsScaler,StandardScaler,SplineTransformer,RobustScaler
from tensorflow.keras.layers import PReLU
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.metrics import R2Score
from optuna.integration import TFKerasPruningCallback
from tensorflow.keras.optimizers import Adam
from optuna.pruners import ThresholdPruner
from sklearn.preprocessing import Normalizer

from functools import partial
from optuna.samplers import TPESampler,QMCSampler,GridSampler,BruteForceSampler,RandomSampler
import optuna
import logging
import sys 


Loading the complete dataset

In [4]:
data=pd.read_json('your_dataset.json')


In [5]:

class MyMinMaxScaler(MinMaxScaler):
    def inverse_transform(self, X):
        X -= self.min_
        X /= self.scale_
        return X

def log10(x):
    return np.log10(x + 1)


def log10_inv(x):
    return (10 ** x) - 1

def get_property_scalers(props):
    scalers = {}
    for propertys in props:
        mm1 = MyMinMaxScaler()
        scalers[propertys] = Pipeline([("minmax", mm1)])
    return scalers

def get_datasets(df_init):
    df = df_init.copy()

    df["scaled_value"] = df["value"]

    props = df['Property'].unique().tolist()

    # Create final dataset for ensemble
    ps_final = get_property_scalers(props)

    for prop in props:
        # train
        cond = df[df['Property'] == prop].index
        val = df.loc[cond, ["scaled_value"]]
        df.loc[cond, ["scaled_value"]] = ps_final[prop].fit_transform(val)

    training_df,val_df=train_test_split(df,test_size=0.2,random_state=0,stratify=df['scala'])
 
    k_fold_datasets = []
    skf = StratifiedKFold(n_splits=5,shuffle=False)
    for train_index, val_index in skf.split(training_df, training_df['scala']):

        k_train = training_df.iloc[train_index].copy()
        k_val = training_df.iloc[val_index].copy()

        k_fold_datasets.append({"train": k_train, "val": k_val})

    return k_fold_datasets,val_df

total,val=get_datasets(data)


Running the trails for tensorflow and optuna

In [6]:
seed = 123
batch_size = 200

np.random.seed(seed)
tf.random.set_seed(seed)

def create_model(trial):

    n_layers = trial.suggest_int("n_layers", 2, 5)
   
    model = tf.keras.Sequential()
    for num in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(num), 64, 512, step=64)
        model.add(tf.keras.layers.Dense(
                num_hidden,
            )) # Adding dense and dropout alternatively

        activation_name = trial.suggest_categorical(
            f"actvtn_l{num}", ['relu','LeakyReLU','selu','gelu', 'prelu']
        )

        if activation_name == "prelu":
            activation_layer=tf.keras.layers.PReLU()      
        elif activation_name == "LeakyReLU":
            activation_layer=tf.keras.layers.LeakyReLU()
        else:
            activation_layer=tf.keras.layers.Activation(activation_name)

        model.add(activation_layer)

        n_drop = trial.suggest_float("n_drop_l{}".format(num), 0, 0.7)
        model.add(tf.keras.layers.Dropout(n_drop))
    
    activation_name_f = trial.suggest_categorical(
            f"final_act", ['relu',None]
        )
    model.add(tf.keras.layers.Dense(1,activation=activation_name_f))
    
    return model

def objective(trial,train_dataset,test_dataset,study_name,validate):

    learning_rate = trial.suggest_float("learning_rate", 5e-4, 5e-3, log=True)
    optimizer = Adam(learning_rate=learning_rate)
    metric = R2Score()
    m=tf.keras.metrics.RootMeanSquaredError(
    name='root_mean_squared_error')

    model = create_model(trial)
    model.compile(optimizer=optimizer, loss="mse", metrics=[metric,m])
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.85, monitor="val_loss", verbose=True
        ),
        tf.keras.callbacks.EarlyStopping(patience=40, monitor="val_loss", verbose=True)
    ]

    fit = model.fit(
        train_dataset,epochs=1000, validation_data=(test_dataset),callbacks=callbacks)
    
    
  
    test_dataset = tf.data.Dataset.from_tensor_slices((list(validate['cfp']),list(validate['scaled_value'])))
  
    test_dataset = (
        test_dataset.cache().batch(50).prefetch(tf.data.experimental.AUTOTUNE)
        )
    
     
    test_metric=model.evaluate(test_dataset)
    
    from pathlib import Path
    pt = Path(f'battery_informatik/{study_name}/{trial.number}.keras')
    pt.parent.mkdir(parents=True, exist_ok=True)
    model.save(pt)
    return test_metric[0]

def objective_cv(trial,num, study_name):
    
    scores = []
    #for 0 in range(len(total)):

    train_input = list(total[num]['train']['cfp'])
    train_target = list(total[num]['train']['scaled_value'])
    test_input = list(total[num]['val']['cfp'])
    test_target = list(total[num]['val']['scaled_value'])

    t_i=test_input
    t_o=test_target

    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_input, train_target)
    )
    test_dataset = tf.data.Dataset.from_tensor_slices((t_i, t_o))

    train_dataset = (
    train_dataset.cache().batch(50).prefetch(tf.data.experimental.AUTOTUNE)
        )  # AUTOTUNE is used to speed up the loading of data to model)
    test_dataset = (
        test_dataset.cache().batch(50).prefetch(tf.data.experimental.AUTOTUNE)
        )
    validate=total[num]['val']
    accuracy = objective(trial,train_dataset=train_dataset,test_dataset=test_dataset,validate=validate,study_name=study_name)
    #scores.append(accuracy)
    return accuracy

In [7]:
from functools import partial
from optuna.samplers import TPESampler,QMCSampler,GridSampler,BruteForceSampler,RandomSampler
import optuna
import logging
import sys 

cv_number = 0
name = "battery_informatik"
name_num = f'battery_informatik_{cv_number}'
p = Path(name_num)
p.parent.mkdir(parents=True, exist_ok=True)
storage_name = "sqlite:///{}.db".format(name_num)

sampler = TPESampler(seed=0)

study = optuna.create_study(study_name=name_num, direction="minimize", storage=storage_name, load_if_exists=True, sampler=sampler)

obj = partial(objective_cv, num=cv_number, study_name=name_num)

study.optimize(obj, n_trials=10000)
               

[I 2025-01-24 10:28:42,553] Using an existing study with name 'battery_informatik_0' instead of creating a new one.


Epoch 1/1000


I0000 00:00:1737710924.656449 2670126 service.cc:145] XLA service 0x7f584c004420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737710924.656490 2670126 service.cc:153]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-01-24 10:28:44.713828: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-24 10:28:44.889309: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/11 ━━━━━━━━━━━━━━━━━━━━ 58s 6s/step - loss: 0.1134 - r2_score: -0.7697 - root_mean_squared_error: 0.3367

I0000 00:00:1737710929.001496 2670126 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1737710929.007291 2670126 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - loss: 0.1169 - r2_score: -0.7192 - root_mean_squared_error: 0.3411

I0000 00:00:1737710933.206616 2670125 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 4 bytes spill stores, 4 bytes spill loads



11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 538ms/step - loss: 0.1158 - r2_score: -0.7053 - root_mean_squared_error: 0.3395 - val_loss: 0.0444 - val_r2_score: 0.3951 - val_root_mean_squared_error: 0.2108 - learning_rate: 0.0018
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0621 - r2_score: 0.0855 - root_mean_squared_error: 0.2490 - val_loss: 0.0458 - val_r2_score: 0.3766 - val_root_mean_squared_error: 0.2140 - learning_rate: 0.0018
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0569 - r2_score: 0.1620 - root_mean_squared_error: 0.2385 - val_loss: 0.0514 - val_r2_score: 0.3006 - val_root_mean_squared_error: 0.2267 - learning_rate: 0.0018
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0525 - r2_score: 0.2260 - root_mean_squared_error: 0.2290 - val_loss: 0.0462 - val_r2_score: 0.3717 - val_root_mean_squared_error: 0.2149 - learning_rate: 0.0018
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0452 - r2_score: 0.3336 - root_mean_squared_error: 

[W 2025-01-24 10:28:57,122] Trial 2 failed with parameters: {'learning_rate': 0.0017692267954532756, 'n_layers': 4, 'n_units_l0': 320, 'actvtn_l0': 'prelu', 'n_drop_l0': 0.6745639323507204, 'n_units_l1': 256, 'actvtn_l1': 'gelu', 'n_drop_l1': 0.06099050979107849, 'n_units_l2': 64, 'actvtn_l2': 'gelu', 'n_drop_l2': 0.32303555357705227, 'n_units_l3': 448, 'actvtn_l3': 'gelu', 'n_drop_l3': 0.2902633579933665, 'final_act': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/subhash/test/batterie_informatik/ne/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_2669479/770857860.py", line 103, in objective_cv
    accuracy = objective(trial,train_dataset=train_dataset,test_dataset=test_dataset,validate=validate,study_name=study_name)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 